# model_R

Date of creation: 13.07.2020

Last updated: 13.07.2020

In [2]:
import math
import numpy as np
import pandas as pd

In [3]:
import NotebookLoader
import Physics as ph
import SSM18 as s

importing Jupyter notebook from Physics.ipynb
Physics version 1.14 13.07.2020
importing Jupyter notebook from SSM18.ipynb
SSM18 version 1.9 13.07.2020


## Интегрирование от центра по расстоянию
параметры модели: плотность и температура в центре.

хим.состав, число точек, шаг по радиусу.

вызывает функцию XYZ(m) для получения кортежа (X, Y, Z) для данной доли массы от полной (m должна быть от 0 до 1)

In [6]:
def model_R(d0, t0, XYZ, Z=0.02, opacity=ph.Opacity, energy=ph.Etot, dim=30000, M=s.M, debug=False) :  
    # distance from the center
    r = np.zeros(dim, dtype='float64')
    # density
    d = np.zeros(dim, dtype='float64')
    # temperature
    t = np.zeros(dim, dtype='float64')
    # luminosity
    l = np.zeros(dim, dtype='float64')
    # mass inside r
    m = np.zeros(dim, dtype='float64')
    # pressure
    p = np.zeros(dim, dtype='float64')
    # hydrogen
    x = np.zeros(dim, dtype='float64')
    # opacity
    k = np.zeros(dim, dtype='float64')
    # 1 - convection, 0 - radiation
    c = np.zeros(dim)
    # скорость выгорания водорода (величина, обратная характерному времени)
    v = np.zeros(dim, dtype='float64')

    X, Y, Z = XYZ(0)
    mu = ph.MU(X, Y, Z)
    R1 = pow(1.5*ph.kB*t0/ph.pi/mu/ph.m_prot/ph.G/d0, 0.5)
    dr = 0.001*R1
    
# начальные данные
    for n in range(2) :
        r[n] = n*dr
        V = 4*ph.pi/3*pow(r[n], 3)
        d[n] = d0*(1 - pow(r[n]/R1, 2))
        t[n] = t0
        m[n] = V*d[n]*(1 - 0.6*pow(r[n]/R1, 2))
        X, Y, Z = XYZ(m[n]/M)
        x[n] = X
        mu = ph.MU(X, Y, Z)
        l[n] = m[n]*energy(d[n], t[n], X, Y, Z)
        p[n] = ph.Pressure(d[n], t[n], X, Y, Z)
        k[n] = opacity(d[n], t[n], X, Y, Z)
        v[n] = 4*ph.m_prot*energy(d[n], t[n], X, Y, Z)/X/ph.dEpp*ph.year
        c[n] = 0.
    for n in range(1, dim - 1) :
        S = 4*ph.pi*r[n]*r[n]
        g = ph.G*m[n]/r[n]/r[n]
        rad = 3/16*k[n]*d[n]*l[n]/ph.sigma/pow(t[n], 3)/S
        con = 0.4*mu/ph.kB*ph.m_prot*g
        r[n+1] = r[n] + dr
        if rad < con:
            c[n+1] = 0.
            dtdr = -rad
        else:
            c[n+1] = 1.
            dtdr = -con
#         dt = 2*dtdr*dr - (t[n] - t[n-1])/(r[n] - r[n-1])*dr
        dt = dtdr*dr
        t[n+1] = t[n] + dt
        dpdr = -g*d[n]
#         p[n+1] = p[n] + 2*dpdr*dr - (p[n] -p[n-1])/(r[n] - r[n-1])*dr
        p[n+1] = p[n] + dpdr*dr
        if p[n+1] < 0. : 
            raise Exception("Negative pressure")
            break
        # Находим плотность из уравнения состояния
        d[n+1] = mu*ph.m_prot*p[n+1]/ph.kB/t[n+1]
        davg = (d[n+1] + d[n])/2
        tavg = (t[n+1] + t[n])/2
        dm = 4*ph.pi/3*(pow(r[n+1], 3) - pow(r[n], 3))*davg
        m[n+1] = m[n] + dm
        l[n+1] = l[n] + dm*energy(davg, tavg, X, Y, Z)
        X, Y, Z = XYZ(m[n+1]/M)
        x[n+1] = X
        mu = ph.MU(X, Y, Z)
        k[n+1] = opacity(d[n+1], t[n+1], X, Y, Z)
        v[n+1] = 4*ph.m_prot*energy(d[n+1], t[n+1], X, Y, Z)/X/ph.dEpp*ph.year
        
        if debug:
            print("%d r=%.4f m=%.4f d=%.3f t=%.0f p=%.2e k=%.2f c=%.1f" % (n, r[n]/s.R, m[n]/M, d[n], t[n], p[n], k[n], c[n]))
        # уменьшаем шаг если температура падает слишком резко
        if abs(dt/t[n]) > 1e-3 :
            dr = 0.7*dr
        if m[n+1]/M > 1.1 : 
            raise Exception('Mass exceeded 1.1*M_target')
        Tsur = pow(l[n]/ph.sigma/S, 1/4)
        if t[n+1] < Tsur :
            break
        
    if (n == dim - 2) :
         raise Exception("Error, increase dim!")

    df = pd.DataFrame({'Mass': m[:n+1], 'Radius': r[:n+1], 'Temperature': t[:n+1], 'Density': d[:n+1], 'Luminosity': l[:n+1], 
                       'Hydrogen': x[:n+1], 'Opacity': k[:n+1], 'Convection': c[:n+1], 'Pressure': p[:n+1], 'Velocity': v[:n+1]})

    return (n, m[n], r[n], l[n], t[n], df)

In [7]:
print('Rmodel version 1.06 16.07.2020')

Rmodel version 1.06 16.07.2020
